In [1]:
import pandas as pd
import numpy as np
import os
import FinanceDataReader as fdr
from sklearn.linear_model import LinearRegression
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

In [2]:
# read stock code, list

path = './open'
list_name = 'Stock_List.csv'
sample_name = 'sample_submission.csv'

stock_list = pd.read_csv(os.path.join(path,list_name))
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x : str(x).zfill(6))
stock_list

,종목명,종목코드,상장시장
0,삼성전자,005930,KOSPI
1,SK하이닉스,000660,KOSPI
2,NAVER,035420,KOSPI
3,카카오,035720,KOSPI
4,삼성바이오로직스,207940,KOSPI
...,...,...,...
371,더네이쳐홀딩스,298540,KOSDAQ
372,코엔텍,029960,KOSDAQ
373,원익홀딩스,030530,KOSDAQ
374,웹케시,053580,KOSDAQ


In [3]:
# select start, end data

start_date = '20210104'
end_date = '20210827'

start_weekday = pd.to_datetime(start_date).weekday()
max_weeknum = pd.to_datetime(end_date).strftime('%V')
Business_days = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])

print(f'WEEKDAY of "start_date" : {start_weekday}')
print(f'NUM of WEEKS to "end_date" : {max_weeknum}')
print(f'HOW MANY "Business_days" : {Business_days.shape}', )
display(Business_days.head())

WEEKDAY of "start_date" : 0
NUM of WEEKS to "end_date" : 34
HOW MANY "Business_days" : (170, 1)


,Date
0,2021-01-04
1,2021-01-05
2,2021-01-06
3,2021-01-07
4,2021-01-08


In [4]:
# read submission file

sample_name = 'sample_submission.csv'
sample_submission = pd.read_csv(os.path.join(path,sample_name))

In [5]:
# public data

model = LinearRegression()
for code in tqdm(stock_list['종목코드'].values):
    data = fdr.DataReader(code, start = start_date, end = end_date)[['Close']].reset_index()
    data = pd.merge(Business_days, data, how = 'outer')
    data['weekday'] = data.Date.apply(lambda x : x.weekday())
    data['weeknum'] = data.Date.apply(lambda x : x.strftime('%V'))
    data.Close = data.Close.ffill()

    # MA
    data['ma5']=data['Close'].ewm(5).mean()
    data['ma20']=data['Close'].ewm(20).mean()

    data_close = pd.pivot_table(data = data, values = 'Close', columns = 'weekday', index = 'weeknum')
    data_ma5 = pd.pivot_table(data = data, values = 'ma5', columns = 'weekday', index = 'weeknum')
    data_ma20 = pd.pivot_table(data = data, values = 'ma20', columns = 'weekday', index = 'weeknum')
    
    # data flatten
    data_close_flatten = np.ravel(data_close.to_numpy())
    data_ma5_flatten = np.ravel(data_ma5.to_numpy())
    data_ma20_flatten = np.ravel(data_ma20.to_numpy())

    predictions = []
    
    # predict day 1

    # close
    data_close_1 = data_close_flatten.reshape((-1,1))
    x = data_close_1[:-6]
    y = data_close_1[1:-5]
    x_close_public = data_close_1[-6]        
    model.fit(x, y)
    prediction_close_1 = model.predict(np.expand_dims(x_close_public,0))  # 51

    # ma5
    data_ma5_1 = data_ma5_flatten.reshape((-1,1))
    x = data_ma5_1[:-6]
    y = data_ma5_1[1:-5]
    x_ma5_public = data_ma5_1[-6]        
    model.fit(x, y)
    prediction_ma5_1 = model.predict(np.expand_dims(x_ma5_public,0))  # 51

    # ma20
    data_ma20_1 = data_ma20_flatten.reshape((-1,1))
    x = data_ma20_1[:-6]
    y = data_ma20_1[1:-5]
    x_ma20_public = data_ma20_1[-6]
    model.fit(x, y)
    prediction_ma20_1 = model.predict(np.expand_dims(x_ma20_public,0))  # 51

    # predict day 1 finish
    
    
    # predict day 2
    
    # close
    data_close_2 = np.append(data_close_flatten, 0)  # padding
    data_close_2 = np.insert(data_close_2, 0, data_close_2[0])
    data_close_2 = data_close_2.reshape((-1,2))  # padding
    x = data_close_2[:-4]
    y = data_close_2[1:-3]
    y_1 = y[:, 1]        
    x_close_public = data_close_2[-4]   
    model.fit(x, y_1)
    prediction_close_2 = model.predict(np.expand_dims(x_close_public,0))  # 51

    # ma5
    data_ma5_2 = np.append(data_ma5_flatten, 0)  # padding
    data_ma5_2 = np.insert(data_ma5_2, 0, data_ma5_2[0])
    data_ma5_2 = data_ma5_2.reshape((-1,2))  # padding
    x = data_ma5_2[:-4]
    y = data_ma5_2[1:-3]
    y_1 = y[:, 1]        
    x_ma5_public = data_ma5_2[-4]   
    model.fit(x, y_1)
    prediction_ma5_2 = model.predict(np.expand_dims(x_ma5_public,0))  # 51

    # ma20
    data_ma20_2 = np.append(data_ma20_flatten, 0)  # padding
    data_ma20_2 = np.insert(data_ma20_2, 0, data_ma20_2[0])
    data_ma20_2 = data_ma20_2.reshape((-1,2))  # padding
    x = data_ma20_2[:-4]
    y = data_ma20_2[1:-3]
    y_1 = y[:, 1]        
    x_ma20_public = data_ma20_2[-4]   
    model.fit(x, y_1)
    prediction_ma20_2 = model.predict(np.expand_dims(x_ma20_public,0))  # 51
    
    # predict day 2 finish
    
    
    # predict day 3
    
    # close
    data_close_3 = np.append(data_close_flatten, 0)  # padding
    data_close_3 = data_close_3.reshape((-1, 3))
    x = data_close_3[:-3]
    y = data_close_3[1:-2]
    y_2 = y[:, 2]        
    x_close_public = data_close_3[-3]
    model.fit(x, y_2)
    prediction_close_3 = model.predict(np.expand_dims(x_close_public,0))
    
    # ma5
    data_ma5_3 = np.append(data_ma5_flatten, 0)  # padding
    data_ma5_3 = data_ma5_3.reshape((-1, 3))
    x = data_ma5_3[:-3]
    y = data_ma5_3[1:-2]
    y_2 = y[:, 2]        
    x_ma5_public = data_ma5_3[-3]
    model.fit(x, y_2)
    prediction_ma5_3 = model.predict(np.expand_dims(x_ma5_public,0))
    
    # ma20
    data_ma20_3 = np.append(data_ma20_flatten, 0)  # padding
    data_ma20_3 = data_ma20_3.reshape((-1, 3))
    x = data_ma20_3[:-3]
    y = data_ma20_3[1:-2]
    y_2 = y[:, 2]        
    x_ma20_public = data_ma20_3[-3]
    model.fit(x, y_2)
    prediction_ma20_3 = model.predict(np.expand_dims(x_ma20_public,0))
    
    # predict day 3 finish

    
    # predict day 4
    
    # close
    data_close_4 = data_close_flatten[1:]
    data_close_4 = np.append(data_close_4, 0)
    data_close_4 = np.append(data_close_4, 0)
    data_close_4 = np.append(data_close_4, 0)
    data_close_4 = data_close_4.reshape((-1, 4))
    x = data_close_4[:-3]
    y = data_close_4[1:-2]
    y_3 = y[:, 3]
    x_close_public = data_close_4[-3]
    model.fit(x, y_3)
    prediction_close_4 = model.predict(np.expand_dims(x_close_public,0))

    # ma5
    data_ma5_4 = data_ma5_flatten[1:]
    data_ma5_4 = np.append(data_ma5_4, 0)
    data_ma5_4 = np.append(data_ma5_4, 0)
    data_ma5_4 = np.append(data_ma5_4, 0)
    data_ma5_4 = data_ma5_4.reshape((-1, 4))
    x = data_ma5_4[:-3]
    y = data_ma5_4[1:-2]
    y_3 = y[:, 3]
    x_ma5_public = data_ma5_4[-3]
    model.fit(x, y_3)
    prediction_ma5_4 = model.predict(np.expand_dims(x_ma5_public,0))

    # ma20
    data_ma20_4 = data_ma20_flatten[1:]
    data_ma20_4 = np.append(data_ma20_4, 0)
    data_ma20_4 = np.append(data_ma20_4, 0)
    data_ma20_4 = np.append(data_ma20_4, 0)
    data_ma20_4 = data_ma20_4.reshape((-1, 4))
    x = data_ma20_4[:-3]
    y = data_ma20_4[1:-2]
    y_3 = y[:, 3]
    x_ma20_public = data_ma20_4[-3]
    model.fit(x, y_3)
    prediction_ma20_4 = model.predict(np.expand_dims(x_ma20_public,0))

    # predict day 4 finish
    
    # predict day 5
    
    # close
    data_close_5 = data_close_flatten.reshape((-1, 5))
    x = data_close_5[:-2]
    y = data_close_5[1:-1]
    y_4 = y[:, 4]        
    x_close_public = data_close_5[-2]
    model.fit(x, y_4)
    prediction_close_5 = model.predict(np.expand_dims(x_close_public,0))

    # ma5
    data_ma5_5 = data_ma5_flatten.reshape((-1, 5))
    x = data_ma5_5[:-2]
    y = data_ma5_5[1:-1]
    y_4 = y[:, 4]        
    x_ma5_public = data_ma5_5[-2]
    model.fit(x, y_4)
    prediction_ma5_5 = model.predict(np.expand_dims(x_ma5_public,0))

    # ma20
    data_ma20_5 = data_ma20_flatten.reshape((-1, 5))
    x = data_ma20_5[:-2]
    y = data_ma20_5[1:-1]
    y_4 = y[:, 4]        
    x_ma20_public = data_ma20_5[-2]
    model.fit(x, y_4)
    prediction_ma20_5 = model.predict(np.expand_dims(x_ma20_public,0))    
    
    # predict day 5 finish
    
    close_rate = 0.5
    ma5_rate = 0.3
    ma20_rate = 0.2
    
    # append
    prediction_ensemble_1 = prediction_close_1*close_rate + prediction_ma5_1*ma5_rate + prediction_ma20_1*ma20_rate
    prediction_ensemble_2 = prediction_close_2*close_rate + prediction_ma5_2*ma5_rate + prediction_ma20_2*ma20_rate
    prediction_ensemble_3 = prediction_close_3*close_rate + prediction_ma5_3*ma5_rate + prediction_ma20_3*ma20_rate
    prediction_ensemble_4 = prediction_close_4*close_rate + prediction_ma5_4*ma5_rate + prediction_ma20_4*ma20_rate
    prediction_ensemble_5 = prediction_close_5*close_rate + prediction_ma5_5*ma5_rate + prediction_ma20_5*ma20_rate
                
    predictions.append(prediction_ensemble_1)
    predictions.append(prediction_ensemble_2)
    predictions.append(prediction_ensemble_3)
    predictions.append(prediction_ensemble_4)
    predictions.append(prediction_ensemble_5)
    
    sample_submission.loc[:4,code] = predictions
sample_submission.isna().sum().sum()

100%|████████████████████████████████████████████████████████████████████████████████| 376/376 [00:37<00:00, 10.07it/s]


0

In [6]:
# Private

start_date = '20210104'
end_date = '20210910'

start_weekday = pd.to_datetime(start_date).weekday()
max_weeknum = pd.to_datetime(end_date).strftime('%V')
Business_days = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])

print(f'WEEKDAY of "start_date" : {start_weekday}')
print(f'NUM of WEEKS to "end_date" : {max_weeknum}')
print(f'HOW MANY "Business_days" : {Business_days.shape}', )
display(Business_days)

WEEKDAY of "start_date" : 0
NUM of WEEKS to "end_date" : 36
HOW MANY "Business_days" : (180, 1)


,Date
0,2021-01-04
1,2021-01-05
2,2021-01-06
3,2021-01-07
4,2021-01-08
...,...
175,2021-09-06
176,2021-09-07
177,2021-09-08
178,2021-09-09


In [7]:
# private data

model = LinearRegression()
for code in tqdm(stock_list['종목코드'].values):
    data = fdr.DataReader(code, start = start_date, end = end_date)[['Close']].reset_index()
    data = pd.merge(Business_days, data, how = 'outer')
    data['weekday'] = data.Date.apply(lambda x : x.weekday())
    data['weeknum'] = data.Date.apply(lambda x : x.strftime('%V'))
    data.Close = data.Close.ffill()
    
    # MA
    data['ma5']=data['Close'].ewm(5).mean()
    data['ma20']=data['Close'].ewm(20).mean()

    data_close = pd.pivot_table(data = data, values = 'Close', columns = 'weekday', index = 'weeknum')
    data_ma5 = pd.pivot_table(data = data, values = 'ma5', columns = 'weekday', index = 'weeknum')
    data_ma20 = pd.pivot_table(data = data, values = 'ma20', columns = 'weekday', index = 'weeknum')
    
    # data flatten
    data_close_flatten = np.ravel(data_close.to_numpy())
    data_ma5_flatten = np.ravel(data_ma5.to_numpy())
    data_ma20_flatten = np.ravel(data_ma20.to_numpy())

    predictions = []

    # predict day 1

    # close
    data_close_1 = data_close_flatten.reshape((-1,1))
    x = data_close_1[:-6]
    y = data_close_1[1:-5]
    x_close_public = data_close_1[-6]
    model.fit(x, y)
    prediction_close_1 = model.predict(np.expand_dims(x_close_public,0))  # 51

    # ma5
    data_ma5_1 = data_ma5_flatten.reshape((-1,1))
    x = data_ma5_1[:-6]
    y = data_ma5_1[1:-5]
    x_ma5_public = data_ma5_1[-6]
    model.fit(x, y)
    prediction_ma5_1 = model.predict(np.expand_dims(x_ma5_public,0))  # 51

    # ma20
    data_ma20_1 = data_ma20_flatten.reshape((-1,1))
    x = data_ma20_1[:-6]
    y = data_ma20_1[1:-5]
    x_ma20_public = data_ma20_1[-6]
    model.fit(x, y)
    prediction_ma20_1 = model.predict(np.expand_dims(x_ma20_public,0))  # 51

    # predict day 1 finish
    
    
    # predict day 2
    
    # close
    data_close_2 = np.append(data_close_flatten, 0)  # padding
    data_close_2 = np.insert(data_close_2, 0, data_close_2[0])
    data_close_2 = data_close_2.reshape((-1,2))  # padding
    x = data_close_2[:-4]
    y = data_close_2[1:-3]
    y_1 = y[:, 1]        
    x_close_public = data_close_2[-4]   
    model.fit(x, y_1)
    prediction_close_2 = model.predict(np.expand_dims(x_close_public,0))  # 51

    # ma5
    data_ma5_2 = np.append(data_ma5_flatten, 0)  # padding
    data_ma5_2 = np.insert(data_ma5_2, 0, data_ma5_2[0])
    data_ma5_2 = data_ma5_2.reshape((-1,2))  # padding
    x = data_ma5_2[:-4]
    y = data_ma5_2[1:-3]
    y_1 = y[:, 1]        
    x_ma5_public = data_ma5_2[-4]   
    model.fit(x, y_1)
    prediction_ma5_2 = model.predict(np.expand_dims(x_ma5_public,0))  # 51

    # ma20
    data_ma20_2 = np.append(data_ma20_flatten, 0)  # padding
    data_ma20_2 = np.insert(data_ma20_2, 0, data_ma20_2[0])
    data_ma20_2 = data_ma20_2.reshape((-1,2))  # padding
    x = data_ma20_2[:-4]
    y = data_ma20_2[1:-3]
    y_1 = y[:, 1]        
    x_ma20_public = data_ma20_2[-4]   
    model.fit(x, y_1)
    prediction_ma20_2 = model.predict(np.expand_dims(x_ma20_public,0))  # 51
    
    # predict day 2 finish
    
    
    # predict day 3
    
    # close
    data_close_3 = data_close_flatten[1:]
    data_close_3 = np.append(data_close_3, 0)  # padding
    data_close_3 = data_close_3.reshape((-1, 3))
    x = data_close_3[:-3]
    y = data_close_3[1:-2]
    y_2 = y[:, 2]
    x_close_public = data_close_3[-3]
    model.fit(x, y_2)
    prediction_close_3 = model.predict(np.expand_dims(x_close_public,0))
    
    # ma5
    data_ma5_3 = data_ma5_flatten[1:]
    data_ma5_3 = np.append(data_ma5_3, 0)
    data_ma5_3 = data_ma5_3.reshape((-1, 3))
    x = data_ma5_3[:-3]
    y = data_ma5_3[1:-2]
    y_2 = y[:, 2]        
    x_ma5_public = data_ma5_3[-3]
    model.fit(x, y_2)
    prediction_ma5_3 = model.predict(np.expand_dims(x_ma5_public,0))
    
    # ma20
    data_ma20_3 = data_ma20_flatten[1:]
    data_ma20_3 = np.append(data_ma20_3, 0)  # padding
    data_ma20_3 = data_ma20_3.reshape((-1, 3))
    x = data_ma20_3[:-3]
    y = data_ma20_3[1:-2]
    y_2 = y[:, 2]        
    x_ma20_public = data_ma20_3[-3]
    model.fit(x, y_2)
    prediction_ma20_3 = model.predict(np.expand_dims(x_ma20_public,0))
    
    # predict day 3 finish

    
    # predict day 4
    
    # close
    data_close_4 = np.append(data_close_flatten, 0)
    data_close_4 = np.append(data_close_4, 0)
    data_close_4 = np.append(data_close_4, 0)
    data_close_4 = np.insert(data_close_4, 0, data_close_4[0])
    data_close_4 = data_close_4.reshape((-1, 4))
    x = data_close_4[:-3]
    y = data_close_4[1:-2]
    y_3 = y[:, 3]
    x_close_public = data_close_4[-3]
    model.fit(x, y_3)
    prediction_close_4 = model.predict(np.expand_dims(x_close_public,0))

    # ma5
    data_ma5_4 = np.append(data_ma5_flatten, 0)
    data_ma5_4 = np.append(data_ma5_4, 0)
    data_ma5_4 = np.append(data_ma5_4, 0)
    data_ma5_4 = np.insert(data_ma5_4, 0, data_ma5_4[0])
    data_ma5_4 = data_ma5_4.reshape((-1, 4))
    x = data_ma5_4[:-3]
    y = data_ma5_4[1:-2]
    y_3 = y[:, 3]
    x_ma5_public = data_ma5_4[-3]
    model.fit(x, y_3)
    prediction_ma5_4 = model.predict(np.expand_dims(x_ma5_public,0))

    # ma20
    data_ma20_4 = np.append(data_ma20_flatten, 0)
    data_ma20_4 = np.append(data_ma20_4, 0)
    data_ma20_4 = np.append(data_ma20_4, 0)
    data_ma20_4 = np.insert(data_ma20_4, 0, data_ma20_4[0])
    data_ma20_4 = data_ma20_4.reshape((-1, 4))
    x = data_ma20_4[:-3]
    y = data_ma20_4[1:-2]
    y_3 = y[:, 3]
    x_ma20_public = data_ma20_4[-3]
    model.fit(x, y_3)
    prediction_ma20_4 = model.predict(np.expand_dims(x_ma20_public,0))

    # predict day 4 finish
    
    # predict day 5
    
    # close
    data_close_5 = data_close_flatten.reshape((-1, 5))
    x = data_close_5[:-2]
    y = data_close_5[1:-1]
    y_4 = y[:, 4]        
    x_close_public = data_close_5[-2]
    model.fit(x, y_4)
    prediction_close_5 = model.predict(np.expand_dims(x_close_public,0))

    # ma5
    data_ma5_5 = data_ma5_flatten.reshape((-1, 5))
    x = data_ma5_5[:-2]
    y = data_ma5_5[1:-1]
    y_4 = y[:, 4]        
    x_ma5_public = data_ma5_5[-2]
    model.fit(x, y_4)
    prediction_ma5_5 = model.predict(np.expand_dims(x_ma5_public,0))

    # ma20
    data_ma20_5 = data_ma20_flatten.reshape((-1, 5))
    x = data_ma20_5[:-2]
    y = data_ma20_5[1:-1]
    y_4 = y[:, 4]        
    x_ma20_public = data_ma20_5[-2]
    model.fit(x, y_4)
    prediction_ma20_5 = model.predict(np.expand_dims(x_ma20_public,0))    
    
    # predict day 5 finish

    
    close_rate = 0.5
    ma5_rate = 0.3
    ma20_rate = 0.2
    
    # append
    prediction_ensemble_1 = prediction_close_1*close_rate + prediction_ma5_1*ma5_rate + prediction_ma20_1*ma20_rate
    prediction_ensemble_2 = prediction_close_2*close_rate + prediction_ma5_2*ma5_rate + prediction_ma20_2*ma20_rate
    prediction_ensemble_3 = prediction_close_3*close_rate + prediction_ma5_3*ma5_rate + prediction_ma20_3*ma20_rate
    prediction_ensemble_4 = prediction_close_4*close_rate + prediction_ma5_4*ma5_rate + prediction_ma20_4*ma20_rate
    prediction_ensemble_5 = prediction_close_5*close_rate + prediction_ma5_5*ma5_rate + prediction_ma20_5*ma20_rate
                
    predictions.append(prediction_ensemble_1)
    predictions.append(prediction_ensemble_2)
    predictions.append(prediction_ensemble_3)
    predictions.append(prediction_ensemble_4)
    predictions.append(prediction_ensemble_5)
    
    sample_submission.loc[5:,code] = predictions
sample_submission.isna().sum().sum()

100%|████████████████████████████████████████████████████████████████████████████████| 376/376 [00:37<00:00,  9.90it/s]


0

In [8]:
sample_submission.to_csv('BASELINE_Linear_Week2.csv',index=False)

In [9]:
sample_submission

,Day,000060,000080,000100,000120,000150,000240,000250,000270,000660,...,330860,336260,336370,347860,348150,348210,352820,357780,363280,950130
0,2021-08-23,23903.513724,31870.194295,59221.003762,167991.967242,89351.203596,17547.569712,55760.901638,82138.338241,105939.560963,...,44569.031360,46172.921952,62989.808344,34096.270428,28986.169874,53082.308331,284171.603827,298298.863792,29249.632844,20217.624673
1,2021-08-24,23759.312913,31778.651069,59641.456786,168173.315620,88608.257588,17484.567923,55762.643496,81999.456414,105925.178671,...,44078.663025,46017.673705,62522.194450,33573.291446,29157.194212,52621.689136,281464.205862,295897.819887,29026.684051,19273.078562
2,2021-08-25,23783.866884,31870.336675,59384.288890,167867.034811,88518.863365,17571.871575,55669.246081,82502.879400,105713.420114,...,43577.544024,45457.692958,62445.745721,33667.892185,29366.365127,53179.651039,278411.508121,294810.156980,29476.808091,18767.911850
3,2021-08-26,23189.345358,31819.700140,59198.394870,168107.063323,86839.975531,17717.417362,55633.365368,82086.201867,105304.696980,...,43305.977934,44604.312060,62442.980084,32644.962584,30056.323058,52546.725166,283302.223748,293870.169472,29044.466061,20373.406779
4,2021-08-27,23424.007986,31758.767651,58640.025696,169013.625117,85488.226249,18527.909681,54769.851439,85511.969158,104135.721013,...,41667.429579,48172.625256,63506.967463,33932.571398,28555.507486,52368.382425,284092.742180,287896.878078,29906.370736,21246.193288
5,2021-09-06,27756.499747,34032.907184,63387.869415,171018.634447,93399.998014,17342.128567,54685.044892,84985.661751,107444.860957,...,47404.906289,51427.945774,61688.021815,35439.559139,33405.456078,57142.548868,284657.008487,293673.194935,29827.534109,18773.612350
6,2021-09-07,28163.567497,34189.748727,63383.835329,171335.225160,93684.633951,17364.813539,54629.669343,85181.798002,107787.536336,...,47625.890199,51550.433003,61779.626765,35695.087914,33391.085966,57727.831902,283865.802912,293936.786140,29736.410134,18761.988646
7,2021-09-08,28272.952918,34100.364081,63514.424576,171531.238475,94132.758056,17466.389234,54921.057391,84097.639203,107879.574220,...,47563.254517,51517.068077,61540.905017,35714.056346,33523.313547,58425.691964,282052.110866,293616.448095,29576.051536,18881.425547
8,2021-09-09,29329.474517,34284.068521,63713.952980,171788.672227,97792.800684,17476.936857,55072.980921,84647.491212,107319.579858,...,47797.599865,52424.602064,61352.581153,35462.234292,33243.878301,57550.241889,284559.049931,294360.045893,29776.743869,18881.603595
9,2021-09-10,28962.921031,34337.920871,63315.550311,172017.203174,92031.073038,17679.675859,54218.223339,84498.632582,106276.910470,...,47780.499258,49328.237795,61132.486279,34341.706222,33888.542269,59574.794668,282322.021290,298182.015245,29947.555152,18234.637125
